<a href="https://colab.research.google.com/github/TanyaGoremykina/Recsystems/blob/main/cw_rec_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install implicit

     |████████████████████████████████| 1.1 MB 8.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606568 sha256=1ab2eb58e50890c0034fd1243485c35dd089fc085f88e7ae37e7486cf7069dae
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
pip install lightfm

     |████████████████████████████████| 310 kB 8.8 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705349 sha256=c8dbe0a6dd7d5e9b4f02991219885e3870de0366dfc039597a493c7fa50a4010
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys

In [4]:
module_path = os.path.abspath(os.path.join(os.pardir))
src_path = os.path.join(module_path, "src")
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(src_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [6]:
data = pd.read_csv('data.csv')
user_features = pd.read_csv('user_features.csv')
item_features = pd.read_csv('item_features.csv')

In [7]:
# схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1.0,1004906.0,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1.0,1033142.0,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [ ]:
recommender = MainRecommender(data_train_lvl_1, item_features, user_features)

In [9]:
users_lvl_2_pred_als = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_pred_als.columns = ['user_id']

In [10]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [11]:
k_n = 5
num = 40

In [12]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
item_list = popularity.sort_values('n_sold', ascending=False).head(num+1).item_id.tolist()
item_list = item_list[1:num+1]
len(item_list)

40

In [13]:
user_train_list = data_train_lvl_1["user_id"].unique().tolist()

In [14]:
def get_pred_als(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [15]:
def get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_tfidf_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [16]:
def get_pred_als_bm25(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_bm25_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [17]:
def get_pred_own(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_own_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [18]:
def get_pred_similar_items(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_similar_items_recommendation(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [ ]:
users_lvl_2_pred_als['predictions'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec"].mean()

In [ ]:
users_lvl_2_pred_als["rec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec"].mean()

In [ ]:
users_lvl_2_pred_als['predictions_tfidf'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_tfidf"].mean()

In [ ]:
users_lvl_2_pred_als["rec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_tfidf"].mean()

In [ ]:
users_lvl_2_pred_als['predictions_bm25'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_bm25(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_bm25"].mean()

In [ ]:
users_lvl_2_pred_als["rec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_bm25"].mean()

In [25]:
users_lvl_2_pred_als.head()

,user_id
0,1385
1,140
2,287
3,909
4,1193


**Строим классификатор**

In [ ]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))

In [27]:
users_lvl_2.head()

,user_id
0,1229
1,314
2,1951
3,253
4,884


In [28]:
users_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  1170 non-null   int64
dtypes: int64(1)
memory usage: 9.3 KB


In [ ]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [31]:
targets_lvl_2.shape

(59600, 3)

In [32]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left') #
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left') #

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,bill_avg,week_last_bill,items_per_week
0,1229,999999.0,1,NaN,NaN,NaN,NaN,NaN,NaN,55-64,A,150-174K,Homeowner,2 Adults No Kids,2,None/Unknown,3.70137,14.0,3.811111
1,1229,859427.0,1,1646.0,PRODUCE,National,SALAD MIX,BLENDS,10 OZ,55-64,A,150-174K,Homeowner,2 Adults No Kids,2,None/Unknown,3.70137,14.0,3.811111


In [33]:
targets_lvl_2.shape

(59600, 19)

In [34]:
targets_lvl_2.fillna(0, inplace=True)

In [35]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [36]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'bill_avg',
 'week_last_bill',
 'items_per_week']

In [37]:
lgb = LGBMClassifier(objective='binary', 
                     n_estimators=1500,
                     learning_rate=0.008,
                     max_depth=7, 
                     verbose=1,
                     categorical_column=cat_feats)

hist = lgb.fit(X_train, y_train, 
               eval_set=[(X_train, y_train)],
               eval_metric=['logloss'],
               early_stopping_rounds=50,
               verbose=0)

train_preds = lgb.predict_proba(X_train)

In [38]:
hist.evals_result_['training']['binary_logloss'][-10:]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [39]:
classifer_prediction = X_train[['user_id', 'item_id']]
classifer_prediction["pred"] = train_preds[:, 1]
classifer_prediction.tail()

,user_id,item_id,pred
59595,999,999999.0,1.000000e-15
59596,999,999999.0,1.000000e-15
59597,999,999999.0,1.000000e-15
59598,999,999999.0,1.000000e-15
59599,999,999999.0,1.000000e-15


In [40]:
res_val = classifer_prediction.loc[classifer_prediction["pred"]>0.5, ["user_id", "item_id"]]
res_val.shape

(0, 2)

In [41]:
pred_lvl_2 = res_val.groupby('user_id')['item_id'].unique().reset_index()
pred_lvl_2.columns=['user_id', 'pred']
pred_lvl_2.head()

,user_id,pred


In [42]:
result_lvl_2 = result_lvl_2.merge(pred_lvl_2, on='user_id', how='left')
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999.0, 865178.0, 953561.0, 995242.0, 10331...",NaN
1,5,"[938983.0, 5980822.0, 999999.0, 825538.0]",NaN
2,7,[999999.0],NaN
3,8,[999999.0],NaN
4,12,"[999999.0, 981760.0, 993638.0, 995965.0, 10408...",NaN


In [43]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["pred"] != result_lvl_2["pred"], 'user_id'].values
len(user_pred_nan)

1612

In [44]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

448

In [45]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

352

In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "pred"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

In [47]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred


In [ ]:
popularity = recommender.prep_data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5 = popularity.sort_values('n_sold', ascending=False).head(6).item_id.tolist()[1:6]
top_5

In [49]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"] =\
        result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "user_id"].apply(lambda x: top_5)

In [50]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred


In [ ]:
len(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])

In [ ]:
res = []
res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])
res

In [53]:
def fill_if_not_5(recommender, x):
    res = []
    res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    pred_num = len(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    if pred_num < 5:
        res.extend(recommender.get_als_recommendations(x, rec_num=5-pred_num))
    return res

In [ ]:
result_lvl_2["pred"] = result_lvl_2["user_id"].apply(lambda x: fill_if_not_5(recommender, x))

In [55]:
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999.0, 865178.0, 953561.0, 995242.0, 10331...",NaN
1,5,"[938983.0, 5980822.0, 999999.0, 825538.0]",NaN
2,7,[999999.0],NaN
3,8,[999999.0],NaN
4,12,"[999999.0, 981760.0, 993638.0, 995965.0, 10408...",NaN


In [ ]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

**LightGBM Ranker**

In [57]:
X_train.shape

(59600, 18)

In [58]:
X_train['num'] = np.arange(len(X_train))

In [59]:
gr = X_train[['user_id', 'num']].groupby('user_id', sort=False).max()['num'].values

In [60]:
grs = np.concatenate((np.array([gr[0]+1]), gr[1:] - gr[:-1]))

In [61]:
X_train.shape, y_train.shape, np.sum(grs)

((59600, 19), (59600, 1), 59576)

In [62]:
from lightgbm import LGBMRanker

lgb_params = { 
    'objective':'lambdarank',
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.08,
    'max_depth': 5,
    'categorical_column': cat_feats,
    'random_state': 27,
    'verbose': 0,
    'is_unbalance': True
}

In [ ]:
lgb_rn = LGBMRanker(**lgb_params, silent=False)

eval_h = lgb_rn.fit(X_train, y_train, group=grs,
                    eval_set=[(X_train, y_train)], eval_group=[grs], 
                    eval_metric=['ndcg'],
                    eval_at=[5, 10 ], 
                    early_stopping_rounds=50, 
                    verbose=0)

In [ ]:
eval_h.evals_result_['training']['ndcg@5'][:3], eval_h.evals_result_['training']['ndcg@5'][-3:]

In [ ]:
rank_preds = lgb_rn.predict(X_train)

In [ ]:
ranker_prediction = X_train[['user_id', 'item_id']]
ranker_prediction["pred"] = rank_preds
ranker_prediction.head()

In [67]:
ranker_prediction = ranker_prediction.drop_duplicates()

In [ ]:
ranker_prediction.sort_values(by="pred", inplace=True, ascending=False)
ranker_prediction.head()

In [69]:
ranked_res = ranker_prediction.groupby('user_id')['item_id'].unique().reset_index()   
ranked_res

,user_id,item_id
0,1,"[999999.0, 840361.0, 845307.0, 852014.0, 94094..."
1,7,"[999999.0, 856335.0, 862682.0, 896369.0, 92155..."
2,8,"[999999.0, 841220.0, 847982.0, 849843.0, 86050..."
3,14,"[999999.0, 5978648.0, 916122.0, 1081189.0, 597..."
4,15,"[826249.0, 999999.0, 845208.0, 848071.0, 85567..."
...,...,...
1165,2487,"[999999.0, 1026118.0, 5566472.0, 5978656.0, 11..."
1166,2488,"[999999.0, 916122.0, 940766.0, 963502.0, 10331..."
1167,2491,"[999999.0, 1005609.0, 1035143.0, 1070105.0, 95..."
1168,2492,"[999999.0, 1042438.0, 1106523.0, 928049.0, 846..."


In [70]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(ranked_res, on='user_id', how='left')
result_lvl_2

,user_id,actual,item_id
0,1,"[999999.0, 865178.0, 953561.0, 995242.0, 10331...","[999999.0, 840361.0, 845307.0, 852014.0, 94094..."
1,5,"[938983.0, 5980822.0, 999999.0, 825538.0]",NaN
2,7,[999999.0],"[999999.0, 856335.0, 862682.0, 896369.0, 92155..."
3,8,[999999.0],"[999999.0, 841220.0, 847982.0, 849843.0, 86050..."
4,12,"[999999.0, 981760.0, 993638.0, 995965.0, 10408...",NaN
...,...,...,...
1607,2493,"[826784.0, 999999.0, 883404.0, 1062966.0, 1106...","[999999.0, 828867.0, 845193.0, 856772.0, 86962..."
1608,2495,"[889731.0, 999999.0]",NaN
1609,2497,"[999999.0, 1037840.0, 5569230.0, 8090537.0, 55...",NaN
1610,2499,"[838186.0, 999999.0, 932949.0, 933835.0, 10676...",NaN


In [71]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["item_id"] != result_lvl_2["item_id"], 'user_id'].values
len(user_pred_nan)

640

In [72]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

448

In [73]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

20

In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "item_id"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

In [75]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,item_id


In [ ]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "item_id"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()